# 🏇 JRA 基本情報スクレイピング (Stage 1/2)

このノートブックは**レース基本情報のみ**を高速取得します。

## 📊 取得データ
- レース結果ページから26カラム
- 1レース = 1リクエスト
- 高速実行(約10秒/レース)

## 🚀 次のステップ
`Colab_JRA_Details.ipynb` で履歴+血統を追加取得

In [ ]:
# Google Drive Mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 🛡️ Keep-Alive (アイドルタイムアウト回避)
from IPython.display import display, Javascript

display(Javascript('''
function ClickConnect(){
    console.log("Keep-alive: Working");
    var buttons = document.querySelectorAll("colab-connect-button");
    buttons.forEach(function(btn){
        btn.click();
    });
}
setInterval(ClickConnect, 60000);
console.log("Keep-alive script started - clicks every 60 seconds");
'''))

print("✅ Keep-alive activated (auto-clicks every 60 seconds)")
print("💡 This prevents idle timeout during long scraping sessions")

In [ ]:
# Settings
SAVE_DIR = '/content/drive/MyDrive/dai-keiba/data/raw'
TARGET_CSV = 'database_basic.csv'  # 基本情報のみ
MASTER_ID_CSV = 'race_ids.csv'

In [ ]:
# Basic Scraping Logic (Race Info Only)
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os
import gc
from tqdm.auto import tqdm

def scrape_race_basic(url, force_race_id=None):
    """
    レース基本情報のみを取得(26カラム)
    履歴・血統は取得しない
    """
    headers = {"User-Agent": "Mozilla/5.0"}
    
    try:
        resp = requests.get(url, headers=headers, timeout=15)
        resp.encoding = 'EUC-JP'
        soup = BeautifulSoup(resp.text, 'html.parser')
        
        # Metadata
        h1_elem = soup.select_one("div.header_line h1 .txt")
        full_text = h1_elem.text.strip() if h1_elem else (soup.h1.text.strip() if soup.h1 else "")
        
        date_text = ""
        match_date = re.search(r'(\d{4}年\d{1,2}月\d{1,2}日)', full_text)
        if match_date:
            date_text = match_date.group(1)
        
        # Initialize variables
        venue_text = ""
        kai = "01"
        day = "01"
        r_num = "10"
        
        if force_race_id:
            race_id = str(force_race_id)
        else:
            # Parse race_id from header
            
            venues_str = "札幌|函館|福島|新潟|東京|中山|中京|京都|阪神|小倉"
            match_meta = re.search(rf'(\d+)回({venues_str})(\d+)日', full_text)
            if match_meta:
                kai = f"{int(match_meta.group(1)):02}"
                venue_text = match_meta.group(2)
                day = f"{int(match_meta.group(3)):02}"
            
            match_race = re.search(r'(\d+)レース', full_text)
            if match_race:
                r_num = f"{int(match_race.group(1)):02}"
            
            place_map = {
                "札幌": "01", "函館": "02", "福島": "03", "新潟": "04", "東京": "05",
                "中山": "06", "中京": "07", "京都": "08", "阪神": "09", "小倉": "10"
            }
            p_code = place_map.get(venue_text, "00")
            year = date_text[:4] if date_text else "2025"
            race_id = f"{year}{p_code}{kai}{day}{r_num}"
        
        # Race Info
        race_name = soup.select_one(".race_name").text.strip() if soup.select_one(".race_name") else ""
        header_text = soup.select_one("div.header_line").text if soup.select_one("div.header_line") else full_text
        
        # Course/Distance
        course_type = ""
        distance = ""
        dt_match = re.search(r'(芝|ダ|ダート|障害)[^0-9]*(\d+)', header_text)
        if dt_match:
            c = dt_match.group(1)
            course_type = '芝' if '芝' in c else ('ダート' if 'ダ' in c else '障害')
            distance = dt_match.group(2)
        
        # Weather/Condition
        weather = ""
        condition = ""
        rotation = ""
        
        w_match = re.search(r'天候\s*[:：]\s*(\S+)', soup.text)
        if w_match:
            weather = w_match.group(1)
        
        c_match = re.search(r'(?:芝|ダート)\s*[:：]\s*(\S+)', soup.text)
        if c_match:
            condition = c_match.group(1)
        
        if "右" in header_text:
            rotation = "右"
        elif "左" in header_text:
            rotation = "左"
        elif "直" in header_text:
            rotation = "直線"
        
        # Parse Result Table
        tables = soup.find_all('table')
        target_table = None
        for t in tables:
            if "着順" in t.text and "馬名" in t.text:
                target_table = t
                break
        
        if not target_table:
            return None
        
        rows = target_table.find_all('tr')
        data_list = []
        
        for row in rows:
            if row.find('th'):
                continue
            cells = row.find_all('td')
            if not cells:
                continue
            
            # Extract basic info
            rank = cells[0].text.strip()
            waku = ""
            if cells[1].find('img'):
                alt = cells[1].find('img').get('alt', '')
                m = re.search(r'枠(\d+)', alt)
                waku = m.group(1) if m else alt
            
            umaban = cells[2].text.strip()
            horse_name_elem = cells[3].find('a')
            horse_name = cells[3].text.strip()
            horse_id = ""
            if horse_name_elem and 'href' in horse_name_elem.attrs:
                hm = re.search(r'/horse/(\d+)', horse_name_elem['href'])
                if hm:
                    horse_id = hm.group(1)
            
            jockey = cells[6].text.strip()
            time_val = cells[7].text.strip()
            
            row_dict = {
                "日付": date_text,
                "会場": venue_text,
                "レース番号": f"{r_num}R",
                "レース名": race_name,
                "重賞": "",
                "コースタイプ": course_type,
                "距離": distance,
                "回り": rotation,
                "天候": weather,
                "馬場状態": condition,
                "着順": rank,
                "枠": waku,
                "馬番": umaban,
                "馬名": horse_name,
                "性齢": cells[4].text.strip(),
                "斤量": cells[5].text.strip(),
                "騎手": jockey,
                "タイム": time_val,
                "着差": cells[8].text.strip(),
                "人気": cells[13].text.strip(),
                "単勝オッズ": cells[14].text.strip() if len(cells) > 14 else "0.0",
                "後3F": cells[10].text.strip(),
                "厩舎": cells[12].text.strip(),
                "馬体重(増減)": cells[11].text.strip(),
                "race_id": race_id,
                "horse_id": horse_id
            }
            data_list.append(row_dict)
        
        df = pd.DataFrame(data_list)
        return df
        
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

print("✅ Basic scraping function loaded")


In [ ]:
# Main Execution
def run_basic_scraping():
    csv_path = os.path.join(SAVE_DIR, TARGET_CSV)
    id_path = os.path.join(SAVE_DIR, MASTER_ID_CSV)
    
    # Load master list
    if not os.path.exists(id_path):
        print(f"❌ マスターリストが見つかりません: {id_path}")
        return
    
    master_df = pd.read_csv(id_path, dtype=str)
    master_ids = set(master_df['race_id'].dropna().unique())
    print(f"📋 マスターリスト全件: {len(master_ids)} レース")
    
    # Load existing
    existing_ids = set()
    if os.path.exists(csv_path):
        existing_df = pd.read_csv(csv_path, dtype=str, usecols=['race_id'])
        existing_ids = set(existing_df['race_id'].dropna().unique())
        print(f"💾 既存取得済み: {len(existing_ids)} レース")
    
    # Calculate diff
    target_ids = sorted(list(master_ids - existing_ids))
    print(f"🚀 今回スクレイピング対象: {len(target_ids)} レース")
    
    if not target_ids:
        print("✅ 全て取得済みです。")
        return
    
    # Scrape
    buffer = []
    chunk_size = 50
    
    for i, rid in enumerate(tqdm(target_ids)):
        url = f"https://race.netkeiba.com/race/result.html?race_id={rid}"
        
        try:
            time.sleep(1)
            df = scrape_race_basic(url, force_race_id=rid)
            
            if df is not None and not df.empty:
                buffer.append(df)
        except Exception as e:
            print(f"  Error: {e}")
        
        # Save chunk
        if len(buffer) >= chunk_size or (i == len(target_ids) - 1 and buffer):
            df_chunk = pd.concat(buffer, ignore_index=True)
            
            if not os.path.exists(csv_path):
                df_chunk.to_csv(csv_path, index=False)
            else:
                df_chunk.to_csv(csv_path, mode='a', header=False, index=False)
            
            print(f"  Saved {len(buffer)} races")
            buffer = []
            gc.collect()
    
    print("✅ スクレイピング完了")

print("✅ Main function ready")

In [ ]:
# Run
run_basic_scraping()